# Clustering Crypto

In [326]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [327]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [328]:
# Keep all the cryptocurrencies that are being traded.
crypto_df.shape

(1252, 7)

In [329]:
# Keep all the cryptocurrencies that have a working algorithm.
# Check for null values

for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column Unnamed: 0 has 0 null values
Column CoinName has 0 null values
Column Algorithm has 0 null values
Column IsTrading has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 508 null values
Column TotalCoinSupply has 0 null values


In [330]:
# Drop IsTrading column
crypto_df.drop(columns=['IsTrading'], inplace = True)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [331]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

In [332]:
# Find duplicate entries
print(f'Duplicate entries: {crypto_df.duplicated().sum()}')

Duplicate entries: 0


In [333]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[(crypto_df['TotalCoinsMined'] != 0)]
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [334]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names = pd.DataFrame(crypto_df['CoinName'])
crypto_names.head()

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum


In [335]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=['CoinName'], inplace = True)
crypto_df.head()

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethash,PoW,1.076842e+08,0


In [336]:
crypto_df.columns = [' ','Algorithm', 'ProofType','TotalCoinsMined', 'TotalCoinSupply']
crypto_df.head()

,,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethash,PoW,1.076842e+08,0


In [337]:
# Set index.
crypto_df.set_index(' ', inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [338]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns = ['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [339]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled)

[[-0.11416167 -0.15072664 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [-0.09006124 -0.142553   -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [ 0.55458069  4.67601177 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 ...
 [-0.11399502 -0.150404   -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [-0.11415874 -0.15071128 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [-0.11367081 -0.14919024 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [340]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
pca

PCA(n_components=3)

In [341]:
crypto_pca = pca.fit_transform(X_scaled)
crypto_pca

array([[-0.33358703,  1.09097908, -0.50820508],
       [-0.3162334 ,  1.09123671, -0.50848073],
       [ 2.41399001,  1.73192407, -0.5341711 ],
       ...,
       [-0.17573864, -2.06010971,  0.45467715],
       [-0.27790382,  0.77164253, -0.22084289],
       [-0.33257631,  1.09094528, -0.50821753]])

In [342]:
# Create a DataFrame with the three principal components.
crypto_pca_df = pd.DataFrame(
    data=crypto_pca, columns=['PC1', 'PC2', 'PC3'], index = crypto_df.index)

crypto_pca_df.head()

,PC1,PC2,PC3
,,,
42,-0.333587,1.090979,-0.508205
404,-0.316233,1.091237,-0.508481
1337,2.413990,1.731924,-0.534171
BTC,-0.136752,-1.345903,0.130609
ETH,-0.130298,-2.082642,0.313350


In [343]:
# Fetch the explained variance for each component
pca.explained_variance_ratio_

array([0.02655348, 0.02025564, 0.01948891])

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [344]:
# Create an elbow curve to find the best value for K.

inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


C:\Users\micha\Anaconda3531\envs\PythonData\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [345]:
# Initialize the K-means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(crypto_pca_df)

# Predict clusters
predictions = model.predict(crypto_pca_df)
predictions

array([3, 3, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 3, 3, 1, 3, 1, 1, 3, 3, 1, 1,
       1, 1, 1, 3, 1, 1, 1, 3, 1, 3, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 3, 3,
       1, 1, 1, 1, 1, 3, 3, 1, 3, 1, 1, 1, 1, 3, 1, 1, 3, 1, 3, 3, 3, 1,
       1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 1, 3, 1, 3, 3, 1, 1, 1, 1, 3,
       3, 1, 3, 1, 1, 3, 3, 1, 3, 3, 1, 1, 3, 3, 1, 3, 3, 1, 3, 1, 3, 1,
       3, 1, 3, 3, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 3,
       1, 3, 1, 1, 3, 1, 3, 1, 3, 3, 1, 1, 3, 1, 1, 3, 3, 1, 3, 1, 3, 3,
       3, 1, 1, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3,
       3, 3, 1, 3, 1, 3, 3, 1, 3, 1, 3, 3, 1, 3, 1, 3, 1, 3, 1, 3, 3, 3,
       3, 1, 3, 3, 3, 3, 3, 1, 1, 3, 3, 1, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3,
       3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 3, 3, 1, 3, 1,
       3, 3, 1, 3, 1, 1, 3, 1, 1, 3, 1, 3, 3, 3, 1, 3, 3, 1, 3, 3, 3, 3,
       3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 1, 3, 1, 3, 1, 1, 1, 1, 3, 1, 3, 3,
       1, 3, 1, 1, 1, 3, 1, 3, 1, 1, 1, 3, 1, 3, 1,

In [346]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
crypto_clustered_df = pd.concat([crypto_df, crypto_pca_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
crypto_clustered_df['CoinName'] = crypto_names['CoinName']

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
crypto_clustered_df['Class'] = model.labels_

# Print the shape of the clustered_df
print(crypto_clustered_df.shape)
crypto_clustered_df.head(10)

(578, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.333587,1.090979,-0.508205,NaN,3
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.316233,1.091237,-0.508481,NaN,3
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.413990,1.731924,-0.534171,NaN,3
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.136752,-1.345903,0.130609,NaN,1
ETH,Ethash,PoW,1.076842e+08,0,-0.130298,-2.082642,0.313350,NaN,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.158105,-1.102403,-0.040199,NaN,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.393873,1.286517,-0.403415,NaN,3
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.149203,-2.250089,0.400293,NaN,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.128678,-2.082732,0.313333,NaN,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [347]:
# Creating a 3D-Scatter with the PCA data and the clusters
# 3D Plot
fig = px.scatter_3d(
    crypto_clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [348]:
# Create a table with tradable cryptocurrencies.
crypto_clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)


:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [349]:
# Print the total number of tradable cryptocurrencies.
rows = len(crypto_clustered_df)
rows

578

In [350]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
crypto_scaled = MinMaxScaler().fit_transform(crypto_clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
crypto_scaled

array([[4.20000000e-11, 5.94230127e-03],
       [5.32000000e-04, 7.00182308e-03],
       [3.14159265e-01, 3.53420682e-02],
       ...,
       [2.10000000e-05, 5.94962775e-03],
       [1.00000000e-06, 5.94243008e-03],
       [1.00000000e-04, 5.96388077e-03]])

In [351]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
new_crypto_scaled = pd.DataFrame(
    data = crypto_scaled,
    columns = ['TotalCoinSupply', 'TotalCoinsMined'],
    index = crypto_clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
new_crypto_scaled['CoinName'] = crypto_clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
new_crypto_scaled['Class'] = crypto_clustered_df['Class']

new_crypto_scaled.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
,,,,
42,4.200000e-11,0.005942,NaN,3
404,5.320000e-04,0.007002,NaN,3
1337,3.141593e-01,0.035342,NaN,3
BTC,2.100000e-05,0.005960,NaN,1
ETH,0.000000e+00,0.006050,NaN,1
LTC,8.400000e-05,0.006006,NaN,1
DASH,2.200000e-05,0.005951,NaN,3
XMR,0.000000e+00,0.005960,NaN,1
ETC,2.100000e-04,0.006056,NaN,1


In [353]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
new_crypto_scaled.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class"
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)